# それぞれのLLMに最適化されたチャット プロンプトをテンプレートから効率的に作成する

❗ **Google Colabで実行する場合、ランタイムのタイプはGPU（T4, V100, A100など）を指定すること**

**予め、Hugging Faceのウェブサイトにおいて、Meta Llama2およびMeta Llama3.1のライセンス許諾に同意し、モデルファイルのダウンロード許可を得る必要があります。**

## 準備

In [ ]:
# Hugging Face Hubにログイン
from huggingface_hub import login
login()

In [ ]:
!python -m pip install --no-cache-dir --upgrade \
transformers \
langchain

## Hugging Face Transformers Templates for Chat Models

In [ ]:
from transformers import AutoTokenizer

In [ ]:
chat = [
    {"role": "system", "content": "あなたは日本語ネイティブで親切なAIアシスタントです。"},
    {"role": "user", "content": "こんにちは。ご機嫌いかがですか？"},
    {"role": "assistant", "content": "とても元気です。あなたのお役にたてることがあれば何なりとお尋ねください。"},
    {"role": "user", "content": "大阪の知人へ贈る、東京の土産を提案してください。"},
]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
tokenizer.use_default_system_prompt = False
prompt_string = tokenizer.apply_chat_template(chat, tokenize=False)
print(prompt_string)

In [ ]:
print(tokenizer.chat_template)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
tokenizer.use_default_system_prompt = False
prompt_string = tokenizer.apply_chat_template(chat, tokenize=False)
print(prompt_string)

## LangChain Prompt Templates

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "あなたは日本語ネイティブで親切なAIアシスタントです。"),
    ("user", "こんにちは。ご機嫌いかがですか？"),
    ("assistant", "とても元気です。あなたのお役にたてることがあれば何なりとお尋ねください。"),
    ("user", "{question}")
])

In [ ]:
prompt_value = prompt_template.invoke({"question": "大阪の知人へ贈る、東京の土産を提案してください。"})
print(prompt_value.to_string())

In [ ]:
msg_list = prompt_value.to_messages()
print(msg_list)

## Templates for Chat ModelsをLangChainで使う

In [ ]:
chat = [
    {"role": "system", "content": "あなたは日本語ネイティブで親切なAIアシスタントです。"},
    {"role": "user", "content": "こんにちは。ご機嫌いかがですか？"},
    {"role": "assistant", "content": "とても元気です。あなたのお役にたてることがあれば何なりとお尋ねください。"},
    {"role": "user", "content": "{question}"},
]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
tokenizer.use_default_system_prompt = False
prompt_string = tokenizer.apply_chat_template(chat, tokenize=False)
print(prompt_string)

In [ ]:
from langchain_core.prompts import PromptTemplate

In [ ]:
prompt_template = PromptTemplate.from_template(prompt_string)
prompt_value = prompt_template.invoke({"question": "大阪の知人へ贈る、東京の土産を提案してください。"})
print(prompt_value.to_string())

## LLMサーバーでテスト

In [ ]:
!python -m pip install --no-cache-dir --upgrade \
langchain-openai \
"llama-cpp-python[server]"

In [ ]:
!huggingface-cli download \
--local-dir models \
mmnga/Llama-3.1-8B-Instruct-gguf \
Llama-3.1-8B-Instruct-Q4_K_S.gguf

In [ ]:
import subprocess

# vLLMサーバーの起動コマンド
command = [
    "python", "-m", "llama_cpp.server",
    "--model", "models/Llama-3.1-8B-Instruct-Q4_K_S.gguf",
    "--n_ctx", "2048"
]

# バックグラウンドで実行
process = subprocess.Popen(command)

# サーバーが起動したことを通知
print("vLLM server is running in the background.")

**以下のような表示が確認できたら次のセルを実行**  
*INFO:     Started server process  
INFO:     Waiting for application startup.  
INFO:     Application startup complete.  
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)*

In [ ]:
!curl --request POST \
    --url http://localhost:8000/v1/chat/completions \
    --header "Content-Type: application/json" \
    --data '{"prompt": "Hello"}'

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [ ]:
llm = ChatOpenAI(
    name="Llama-3.1-8B-Instruct-Q4_K_S.gguf",
    base_url="http://localhost:8000/v1",
    api_key="DUMMY"
)

In [ ]:
output_parser = StrOutputParser()
chain = prompt_template | llm | output_parser

In [ ]:
for s in chain.stream({"question": "大阪の知人へ贈る、東京の土産を提案してください。"}):
    print(s, end="", flush=True)

In [ ]:
# サーバーを停止する
process.terminate()
print("vLLM server has been stopped.")